In [51]:
# !pip install --user xmltodict

In [1]:
import random
import pytz
import shapely
from shapely.geometry import Point
import elasticsearch
from datetime import datetime
import time


In [2]:
# define mapping
mapping = {
        "mappings": {
            #"mydocs": {
                "properties": {
                    "Date": {
                     "type": "date"
                    },
                    "Destination": {
                     "type": "geo_point"
                    },
                    "Origin": {
                      "type": "geo_point"
                    },
                    "Title": {
                      "type": "keyword"
                    },
                }
            #}
       }
}

In [3]:
# load Elasticsearch
try:
    es = elasticsearch.Elasticsearch(["kooplex-fiek.elte.hu"],port=9100)
except:
    print( "unable to es")

In [4]:
# create index mass_traffic
try:
    response = es.indices.create(index="random_point",body=mapping, ignore=400)
except Exception as e:
    print(e)

In [5]:
response

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [random_point/qtb5UihITvKYhDahdYH0VA] already exists',
    'index_uuid': 'qtb5UihITvKYhDahdYH0VA',
    'index': 'random_point'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [random_point/qtb5UihITvKYhDahdYH0VA] already exists',
  'index_uuid': 'qtb5UihITvKYhDahdYH0VA',
  'index': 'random_point'},
 'status': 400}

In [6]:
now = pytz.utc.localize(datetime.utcnow())
id=1
latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]
while True:
    try:
        new_body = {}
        new_body["Date"] =  datetime.strftime(now,'%Y-%m-%dT%H:%M:%S.%fZ')
        p = Point((random.uniform(0.2+latr1,23+latr1), random.uniform(45.5+lonr1,49+lonr1))) 
        new_body["Destination"] = { "lat": p.x, 'lon': p.y}                
        p = Point((random.uniform(-43+latr2,-10+latr2), random.uniform(25.5+lonr2,39+lonr2))) 
        new_body["Origin"] = {"lat": p.x, 'lon': p.y}              

        new_body["Title"] = random.choice(['flight1, flight2, flight3'])
        id+=1


        print(new_body)
        res = es.index(index="random_point", doc_type="_doc", id=id, body=new_body)
        print( "------------------------------------------")
        print(res)
    except Exception as e:
        print( "------------------------------------------")
        print("unable to index in elasticsearch")
        print( "------------------------------------------")
        error = "{0}: {1}".format(id,e)
        print( error)
        print( "------------------------------------------")
    print( "all done, starting again soon")
    time.sleep(5)
    if id==30:
        id=1
        latr1, latr2, lonr1, lonr2 = random(4)
        latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]


{'Date': '2020-11-18T22:22:46.992122Z', 'Destination': {'lat': 2.446666040740263, 'lon': 39.691025283524425}, 'Origin': {'lat': -40.997995434117186, 'lon': 33.4908122609994}, 'Title': 'flight1, flight2, flight3'}
------------------------------------------
{'_index': 'random_point', '_type': '_doc', '_id': '2', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 245, '_primary_term': 1}
all done, starting again soon
{'Date': '2020-11-18T22:22:46.992122Z', 'Destination': {'lat': 15.635578954524437, 'lon': 40.177133412237296}, 'Origin': {'lat': -15.880408809584328, 'lon': 26.700175268293563}, 'Title': 'flight1, flight2, flight3'}
------------------------------------------
{'_index': 'random_point', '_type': '_doc', '_id': '3', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 246, '_primary_term': 1}
all done, starting again soon


KeyboardInterrupt: 

## Queries DSL

GET random_point/_search
{
   "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "Origin": {
            "top_left": {
              "lat": -28.6603547,
              "lon": 0.2268691
            },
            "bottom_right": {
              "lat": -48.3658642,
              "lon": -26.9556643
            }
          }
        }
      }
    }
  }
}



In [ ]:
res=es.get(index='mass_traffic',doc_type='record',id='massdot1-daeaf7570000')
res

In [41]:
res= es.search(index='mass_traffic',body={'query':{'match_all':{}}})

In [42]:
res['hits']['total']

{'value': 222, 'relation': 'eq'}

In [43]:
res= es.search(index='mass_traffic',body={'query':{'match':{'Status':'Active'}}})

In [44]:
res

{'took': 148,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'failed': 0},
 'hits': {'total': 32,
  'max_score': 0.087011375,
  'hits': [{'_index': 'mass_traffic',
    '_type': 'record',
    '_id': 'massdot1-0e0d21570000',
    '_score': 0.087011375,
    '_source': {'LastUpdated': '2020-11-18T19:33:40',
     'PairID': 'massdot1-0e0d21570000',
     'Destination': 'Yankee Division Hwy after Exit 20B in Beverly',
     'FreeFlow': 55.0,
     'Origin': 'Yankee Division Hwy before Exit 21 in Beverly',
     'Speed': 109.0,
     'Status': 'Active',
     'Title': 'Yankee Division Hwy before Exit 21 in Beverly to Yankee Division Hwy after Exit 20B in Beverly',
     'TravelTime': 27.0,
     'dnid': '3086',
     'onid': '3063',
     'Route': {'type': 'LineString',
      'coordinates': [[-70.8937460184082, 42.5743799003251],
       [-70.8947438001617, 42.5742297866267],
       [-70.8960580825794, 42.5740559703139],
       [-70.8982092142091, 42.5737320386199],
       [-70.90041935443